## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Rikitea,PF,-23.1203,-134.9692,71.02,80,80,13.27,moderate rain,2022-06-29 23:01:38
1,1,Deputatskiy,RU,69.3000,139.9000,53.46,39,93,4.23,overcast clouds,2022-06-29 23:01:39
2,2,Teguldet,RU,57.3064,88.1667,59.77,89,100,7.25,overcast clouds,2022-06-29 23:01:39
3,3,Port Alfred,ZA,-33.5906,26.8910,62.40,38,11,7.27,few clouds,2022-06-29 23:01:39
4,4,Najran,SA,17.4924,44.1277,79.43,23,61,4.38,broken clouds,2022-06-29 23:01:40


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 85


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Rikitea,PF,-23.1203,-134.9692,71.02,80,80,13.27,moderate rain,2022-06-29 23:01:38
4,4,Najran,SA,17.4924,44.1277,79.43,23,61,4.38,broken clouds,2022-06-29 23:01:40
5,5,Hithadhoo,MV,-0.6000,73.0833,81.95,77,62,8.41,broken clouds,2022-06-29 23:01:40
6,6,Natchez,US,31.5604,-91.4032,75.42,94,40,8.05,moderate rain,2022-06-29 23:01:41
10,10,Ribeira Grande,PT,38.5167,-28.7000,68.40,88,75,3.44,broken clouds,2022-06-29 23:01:42
11,11,Chateaubelair,VC,13.2833,-61.2500,78.19,84,100,17.40,overcast clouds,2022-06-29 23:01:43
13,13,Georgetown,MY,5.4112,100.3354,76.93,94,20,0.00,moderate rain,2022-06-29 23:00:18
16,16,Norman Wells,CA,65.2820,-126.8329,74.19,49,92,12.91,light rain,2022-06-29 23:01:44
18,18,Southbridge,US,42.0751,-72.0334,79.56,60,38,1.01,scattered clouds,2022-06-29 23:01:47
19,19,Balaipungut,ID,1.0500,101.2833,71.65,94,99,3.53,overcast clouds,2022-06-29 23:01:47


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Date                   0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,71.02,moderate rain,-23.1203,-134.9692,
4,Najran,SA,79.43,broken clouds,17.4924,44.1277,
5,Hithadhoo,MV,81.95,broken clouds,-0.6000,73.0833,
6,Natchez,US,75.42,moderate rain,31.5604,-91.4032,
10,Ribeira Grande,PT,68.40,broken clouds,38.5167,-28.7000,
11,Chateaubelair,VC,78.19,overcast clouds,13.2833,-61.2500,
13,Georgetown,MY,76.93,moderate rain,5.4112,100.3354,
16,Norman Wells,CA,74.19,light rain,65.2820,-126.8329,
18,Southbridge,US,79.56,scattered clouds,42.0751,-72.0334,
19,Balaipungut,ID,71.65,overcast clouds,1.0500,101.2833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [12]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,71.02,moderate rain,-23.1203,-134.9692,People ThankYou
4,Najran,SA,79.43,broken clouds,17.4924,44.1277,محمد هادي ال فطيح
5,Hithadhoo,MV,81.95,broken clouds,-0.6000,73.0833,Scoop Guest House
6,Natchez,US,75.42,moderate rain,31.5604,-91.4032,Hampton Inn & Suites Natchez
10,Ribeira Grande,PT,68.40,broken clouds,38.5167,-28.7000,Quinta da Meia Eira
...,...,...,...,...,...,...,...
701,Planeta Rica,CO,78.76,scattered clouds,8.4147,-75.5883,HOTEL MEDIA LUNA
702,Augusto Correa,BR,75.09,clear sky,-1.0217,-46.6350,Fazenda Bacuri
703,Lewisporte,CA,71.31,few clouds,49.2499,-55.0482,Ocean View Bed & Breakfast
706,Puerto Santander,CO,78.21,overcast clouds,8.3636,-72.4063,Hotel Toronto


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))